In [1]:
from pymongo import MongoClient
import openai
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Set up function to create embeddings
# Code snippet copied from https://www.mongodb.com/products/platform/atlas-vector-search
openai.api_key = os.getenv("OPENAI_API_KEY")

model = "text-embedding-ada-002"

def generate_embedding(text: str) -> list[float]:
	resp = openai.Embedding.create(
		input=[text], 
		model=model)
	return resp["data"][0]["embedding"] 

In [12]:
# Set up mongodb connection
client = MongoClient(os.environ['MONGODB_URI'])
db = client["vectordb-test"]
collection = db['sevilla']

In [30]:
query = "Que son los platos mas tipicos de Sevilla?"

In [31]:
# https://www.mongodb.com/docs/atlas/atlas-search/knn-beta/

results = collection.aggregate([
  {
    "$search": {
      "index": "semantic-search",
      "knnBeta": {
        "vector": generate_embedding(query),
        "path": "vector",
        "k": 5
      }
    }
  }
])

In [32]:
for i, document in enumerate(results, start=1):
    print(f"Result {i}):")
    print(document.get('text'))
    print(''.join(['-'] * 50))
    print("\n")

Result 1):
La cultura del tapeo va desde la muestra de guisos y platos calientes pasando por fritos, arroces y guisos, hasta la más ligera de las tapas frías, aliños y chacinas, así como las aceitunas sevillanas en sus variedades encurtidas o aliñadas: gordales, manzanillas, machacadas, entre otras.[299]​
Entre los dulces típicos tradicionales sevillanos que forman parte de la repostería andaluza, destacan la torta de aceite, las tortas de polvorón, los pestiños, el alfajor, las yemas de San Leandro y el tocino de cielo.
--------------------------------------------------


Result 2):
Al igual que en toda Andalucía, destacan las tapas.
--------------------------------------------------


Result 3):
En los meses invernales es típico de la gastronomía autóctona el cocido andaluz compuesto por garbanzos y pringá (distintos embutidos, carne y tocino), las espinacas con garbanzos, la cola de toro, característica en la época de corridas de toros, el menudo o callos, los huevos a la flamenca, 